In [58]:
import pandas as pd
from collections import defaultdict
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)




In [59]:
# 'player_play' as dataframe
player_play = pd.read_csv('data/player_play.csv')

# filtered 'player_play' for only rows with player in motion
player_play_motion_only = player_play[player_play['motionSinceLineset'] == 1]

# store motion play's gameId and playId
motion_play_ids = player_play_motion_only[['gameId', 'playId']].values.tolist()
motion_play_ids_set = set(map(tuple, motion_play_ids))

# 'motion_plays' to for all plays that included motion
motion_plays = player_play[player_play[['gameId', 'playId']].apply(tuple, axis=1).isin(motion_play_ids_set)]
print('Total plays with motion: ' + str(len(motion_plays)))

# 'no_motion_plays' for all plays that did not include motion
no_motion_plays = player_play[~player_play[['gameId', 'playId']].apply(tuple, axis=1).isin(motion_play_ids_set)]
print('Total plays without motion: ' + str(len(no_motion_plays)))


Total plays with motion: 108636
Total plays without motion: 246091


In [60]:
def calcBooleanRate(df, colName):
    
    occurrances = df[df[colName] == 1]
    occurance_play_ids = occurrances[['gameId', 'playId']].values.tolist()
    occurance_play_ids = set(map(tuple, occurance_play_ids))
    
    all_play_ids = df[['gameId', 'playId']].values.tolist()
    all_play_ids = set(map(tuple, all_play_ids))
    
    return len(occurance_play_ids)/len(all_play_ids)

print('Pressure rate, plays with motion ' + str(calcBooleanRate(motion_plays, 'causedPressure')))
print('Pressure rate, plays without motion ' + str(calcBooleanRate(no_motion_plays, 'causedPressure')))
    

Pressure rate, plays with motion 0.222154718509518
Pressure rate, plays without motion 0.19408188807437868


In [61]:
# Define motion man for motion_plays dataframe
player_play_motion_only = motion_plays[motion_plays['motionSinceLineset'] == 1]

# store motion play's gameId and playId and nflId
motion_nfl_ids = player_play_motion_only[['gameId', 'playId','nflId']].values.tolist()

# account for duplicates (plays with more than one man in motion)
motion_nfl_ids_dict = defaultdict(list)

for entry in motion_nfl_ids:
    key = (entry[0], entry[1])  # First two items as the key
    motion_nfl_ids_dict[key].append(entry[2])  # Append the third item

# Add motion player back into ALL rows of motion_plays
motion_plays.loc[:, 'playersInMotion'] = motion_plays.apply(
    lambda row: motion_nfl_ids_dict.get((row['gameId'], row['playId']), None), 
    axis=1
)


In [62]:
# 'players' as dataframe
players = pd.read_csv('data/players.csv')

# Create a dictionary for quick lookup of positions by nflId
position_lookup = dict(zip(players['nflId'], players['position']))

# Add a new column to df1 that maps playersInMotion to positions
motion_plays.loc[:, 'positionsInMotion'] = motion_plays['playersInMotion'].apply(
    lambda players: [position_lookup.get(player, None) for player in players]
)

# Flatten the lists in 'positionsInMotion' and get unique values
unique_positions = set(position for positions in motion_plays['positionsInMotion'] for position in positions)

# Print the unique positions
print(unique_positions)

# Print amount of plays with different combinations of motion man
print(motion_plays['positionsInMotion'].value_counts())



{'TE', 'QB', 'FB', 'WR', 'RB'}
positionsInMotion
[WR]                    41316
[TE]                    24684
[RB]                    23738
[FB]                     3718
[TE, WR]                 2090
                        ...  
[RB, FB, WR]               22
[TE, TE, RB, TE]           22
[WR, WR, WR, TE, RB]       22
[TE, RB, WR, WR]           22
[TE, RB, TE, WR, TE]       22
Name: count, Length: 81, dtype: int64


In [76]:
# Recombine motion_plays and no_motion_plays
no_motion_plays['playersInMotion'] = [['None'] for _ in range(len(no_motion_plays))]
no_motion_plays['positionsInMotion'] = [['None'] for _ in range(len(no_motion_plays))]

player_play = pd.concat([motion_plays, no_motion_plays], ignore_index=True)
player_play['positionsInMotionStr'] = player_play['positionsInMotion'].apply(lambda x: ','.join(x))

# Find unique combinations (preserving the lists as they are)
positionsInMotion = player_play['positionsInMotionStr'].unique().tolist()
print(positionsInMotion)


['WR', 'RB', 'FB', 'TE', 'TE,WR,WR', 'WR,RB', 'WR,WR,RB', 'TE,WR', 'WR,WR,TE,RB,WR', 'WR,WR', 'WR,TE', 'WR,WR,TE,RB', 'TE,TE', 'RB,TE,WR', 'RB,TE', 'QB', 'FB,TE', 'FB,WR', 'RB,TE,TE', 'TE,TE,WR', 'TE,WR,TE', 'QB,TE,FB', 'RB,FB', 'TE,RB', 'RB,WR', 'TE,TE,RB', 'WR,TE,WR', 'TE,FB', 'WR,WR,TE,WR,RB', 'WR,WR,RB,TE,WR', 'WR,RB,WR', 'WR,WR,WR', 'WR,TE,RB', 'TE,RB,TE', 'TE,RB,WR', 'RB,WR,WR,WR', 'RB,WR,WR', 'RB,TE,TE,TE', 'WR,FB', 'RB,WR,TE', 'RB,RB', 'WR,TE,RB,WR,WR', 'QB,FB', 'TE,WR,RB', 'RB,WR,WR,TE', 'WR,RB,TE,WR,WR', 'TE,RB,WR,TE,TE', 'TE,WR,TE,RB', 'FB,WR,TE', 'WR,WR,TE', 'FB,RB', 'RB,WR,TE,TE', 'RB,TE,WR,TE,TE', 'WR,TE,TE,WR', 'WR,RB,TE,TE', 'TE,RB,TE,TE', 'WR,TE,WR,WR,RB', 'WR,RB,TE', 'WR,RB,TE,WR,TE', 'WR,TE,TE', 'WR,WR,RB,WR,TE', 'WR,RB,TE,WR', 'WR,FB,WR', 'WR,TE,TE,WR,RB', 'WR,TE,RB,WR,RB', 'FB,WR,TE,TE,RB', 'TE,TE,TE', 'RB,WR,TE,FB,WR', 'WR,TE,FB,WR', 'WR,TE,RB,WR', 'WR,FB,RB', 'RB,RB,TE,RB,TE', 'WR,TE,WR,RB', 'WR,RB,TE,TE,TE', 'WR,RB,WR,TE', 'WR,RB,WR,WR,TE', 'RB,FB,WR', 'TE,TE,RB

In [77]:
# Find pressure rate for each combo of motion positions
for combo in positionsInMotion:
    print("Positions in motion: " + str(combo) + " , Pressure rate: " + 
          str(calcBooleanRate(player_play[player_play['positionsInMotionStr'] == combo], 'causedPressure')))

Positions in motion: WR , Pressure rate: 0.22949946751863684
Positions in motion: RB , Pressure rate: 0.2020389249304912
Positions in motion: FB , Pressure rate: 0.1952662721893491
Positions in motion: TE , Pressure rate: 0.22103386809269163
Positions in motion: TE,WR,WR , Pressure rate: 0.0
Positions in motion: WR,RB , Pressure rate: 0.23809523809523808
Positions in motion: WR,WR,RB , Pressure rate: 0.3333333333333333
Positions in motion: TE,WR , Pressure rate: 0.16842105263157894
Positions in motion: WR,WR,TE,RB,WR , Pressure rate: 0.5
Positions in motion: WR,WR , Pressure rate: 0.3387096774193548
Positions in motion: WR,TE , Pressure rate: 0.1625
Positions in motion: WR,WR,TE,RB , Pressure rate: 0.0
Positions in motion: TE,TE , Pressure rate: 0.13333333333333333
Positions in motion: RB,TE,WR , Pressure rate: 0.0
Positions in motion: RB,TE , Pressure rate: 0.26666666666666666
Positions in motion: QB , Pressure rate: 0.2727272727272727
Positions in motion: FB,TE , Pressure rate: 0.1
P